In [1]:
import scipy.io
import numpy as np
from __future__ import division
from sklearn.model_selection import LeaveOneOut as loo
from sklearn.metrics import accuracy_score, classification_report
from hmmlearn.hmm import GaussianHMM
# hmmlearn is a library for machine learning with Hidden Markov Models
# At the moment of coding, I had a lot of warnings for deprecation of commands in the next updates, to hide them:
import warnings 
warnings.simplefilter("ignore")

# Import the data
mat = scipy.io.loadmat('HAR_database.mat')
# Extract x_train and y_train
train = mat['database_training']
x_train = train[:,0]
y_train = train[:,1]
# Extract x_test (unlabeled)
x_test = mat['database_test'][:,0]

The train partition is composed by 8 labelled sequences, each one represent the
records for a human being. Each sequence is composed by 5-dimensional signals
corresponding to the person performing a set of activities (running, walking,
standing, sitting, lying) in seminaturalistic conditions. The first 2 dimensions
concern data from an Accelerometer: the z-axis and the modulus of the (x,y)
axes. The following three refer to a 3-axes gyroscope. The lenght of a sequence
representing a person varies from one to another, in a range from approximately
16 thousand observations, to 19 thousand. Each observation (sampled with a
16Hz frequency) is labelled with a number from 1 to 5:

1. walking

2. running

3. standing

4. sitting

5. lying

In [2]:
# show the first two people
x_train[0:2]

array([ array([[ 0.02417325,  0.01990478,  0.03474859, ...,  0.15302195,
         0.16644707,  0.13097667],
       [ 0.59441693,  0.60247182,  0.52582067, ...,  1.9454901 ,
         2.00124793,  1.9890223 ],
       [-0.02273627, -0.01287489, -0.0200163 , ...,  0.00468276,
        -0.00249675, -0.00409452],
       [ 0.11196179,  0.10379559,  0.10319319, ...,  0.11359097,
         0.10181863,  0.10699662],
       [ 0.06049883,  0.05515676,  0.05754055, ...,  0.04945955,
         0.06203927,  0.06837131]]),
       array([[-0.00614456,  0.09416632,  0.02556427, ...,  0.25846636,
         0.26831659,  0.20932986],
       [ 0.15163056,  0.27388097,  0.24967994, ...,  1.01129501,
         1.00222655,  0.94156865],
       [-0.02664025, -0.02277741, -0.00710769, ..., -0.02000222,
        -0.0194218 , -0.01871779],
       [-0.06663789, -0.08298231, -0.10177238, ..., -0.03517126,
        -0.04314523, -0.04061162],
       [ 0.24180082,  0.23684254,  0.24130693, ...,  0.24131465,
         0.2341493

A straightforward approach is to build a Hidden Markov Model for each one of
the 5 activities. Therefore, at the end of the training procedure, I will have 5
different models. With this, a way to predict the activity for the observations in
the test sets (or in a new records), will simply be to compare the log-likelihood
of the models on each 5-dimensional signal in the sequential records for a person.

Given that activities are different, the sequence of 5-dim signals recorded during
the times that activity is performed will have a different shape than signals
recorded in other moments. For example I could intuitively expect a wave with
larger oscillation for an activity as running, with respect to the others; probably I
will need an higher number of hidden states to describe it. Moreover I recall that
we are studying people, not robot, hence I have to consider how each person "do
activities" in a unique way; I expect divergencies in nearly every activity-related
meausre: the rythm, how they balance their bodies, how they breath, even if
sometimes it is just a slight divergence. Because of that, when I train an HMM
on a set of people (especially when it is a reduced as our train partition with 8
individuals) I guess it is importan to assume they can be either very different
from each other or overhall similar except of an anomal person, or whichever
other combination. Therefore the hidden-states tuning has to be conducted with
a certain care: I deploy a LOO cross-validation approach, adapted in this case
as a Leave-one-(person)-out. Precisely, what does the following function do?

I can provide it with the label of an action (1,2,3,4,5 in this case) and the
number of hidden states I want to test (if I give for example, 15, it means
every int from 1 to 15).

+ I use the function loo from the scikitlearn.crossvalidation library to automatically compute leave-lone-out cross-validation indices for the 8 people
in the train dataset.
+ From now on, it is a structure of iterations into iterations:
+ For each hidden-state I want to try:
+ For each pair of indices (ex: train [0,1,2,3,4,5,6],validation[7]) yielded by the leave-one-out function:
+ >> I split the original train dataset: 7 people as the (sub)train partition
and I keep one out for validation.
+ >> From each person’s record in the (sub)train partition I extract only
the singals belonging to the action under study and I concatenate
them into a unique array.
+ >> On this array I fit a HMM with Gaussian emission, diagonal covariance matrix, and the tried number of hidden states.
+ >> I compute the log-likelihood of that HMM on the person representing the validation partition (obviously this was as well subsetted only for the signals belonging to the analyzed action). This log-likelihood is stored.
+ >> At the end of the cross-validation of the HMMs for that hiddenstate I will have 8 different log-likelihood scores, each one yielded by the model fitted on 7 people and validated on the left-out person. I compute the average of them and consider it a reasonable benchmark for the success of an HMM with that number of hidden-states, in the prediction task for the action under consideration.
+ Once I’ve repeated the procedure for all the different hidden-states desired to test, I can confront the (average) log-likelihood yielded by each one of them during the LOO cross-validation, and choose the number of hiddenstates that returns the maximum one.

The important aspect of this approach is that, using the Leave-one-out approach, the log-likelihood scores I obtain from each iteration are very sensitive
to the particularity of the person I’m testing on. This will be evidenced soon commenting my results.

***
### Leave-One-Out cross-validation

In [13]:
def LOOcv(action, hs, num_iter):
    range_hiddenStates = range(1,hs+1)
    global hs_actions
    hs_actions= {1:0,2:0,3:0,4:0,5:0}
    LOO_lls = [] # to store the avg log-likelihoods obtained throughout the LOOcv trying hidden-states = hs
    # for each hidden-state
    for hs in range_hiddenStates:
        # to store the log-likelihood with 'hs' hidden states, obtained at each validation in the LOOcv
        lls_hs=[]
        # leave-one-(person)-out split
        people = range(0,8)
        splits = loo().split(people)
        for people_train, person_val in splits:
            # train
            action_train = np.empty([0,5]) #5 is the num of dimensions for each obs 
            seqlenght_train = np.empty(0)
            for person in people_train:    
                action_ix = [i for i,j in enumerate(y_train[person][0]) if j == action]
                person_action = x_train[person][:,action_ix]
                action_train = np.append(action_train, np.transpose(person_action), axis=0)
                seqlenght_train = np.append(seqlenght_train, person_action.shape[1])
            # validation
            action_val = np.empty([0,5]) #5 is the num of dimensions for each obs 
            seqlenght_val = np.empty(0)
            val_action_ix = [i for i,j in enumerate(y_train[person_val[0]][0]) if j == action]
            val_person_action = x_train[person_val[0]][:,val_action_ix]
            action_val = np.append(action_val, np.transpose(val_person_action), axis=0)
            seqlenght_val = np.append(seqlenght_val, action_val.shape[1])
            # GaussianHMM
            model = GaussianHMM(n_components=hs,
                                covariance_type="diag",
                                n_iter= num_iter).fit(action_train, seqlenght_train.astype(int))
            # store the log-likelihood of the model on the validation partition
            lls_hs.append(model.score(action_val, seqlenght_val))
        # the score with n hidden-states is an average of the log-likelihoods obtained with the LOOcv 
        print 'Action %s: log-likelihood with %s hidden states = %s' %(action, hs, np.mean(lls_hs))
        # I append this mean to LOO_lls
        LOO_lls.append(np.mean(lls_hs))
    # Finally I will choose the number of hidden states that returns the highest avg(ll)
    print 'The best number of hidden-states for action %s is: %s' %(action, LOO_lls.index(max(LOO_lls))+1)
    hs_actions[action] = LOO_lls.index(max(LOO_lls))+1
    return hs_actions

In [14]:
from joblib import Parallel, delayed
import multiprocessing
     
num_cores = multiprocessing.cpu_count()

actions = range(1,6)
    
Parallel(n_jobs=num_cores)(delayed(LOOcv)(action, 5, 10000) for action in actions)

Action 1: log-likelihood with 1 hidden states = -63.4932376436
Action 2: log-likelihood with 1 hidden states = -31.0731412366
Action 3: log-likelihood with 1 hidden states = -0.363394588714
Action 4: log-likelihood with 1 hidden states = -6.93741697547
Action 1: log-likelihood with 2 hidden states = -59.8215499662
Action 4: log-likelihood with 2 hidden states = 0.929071144164
Action 3: log-likelihood with 2 hidden states = 4.68990214363
Action 1: log-likelihood with 3 hidden states = -57.455918976
Action 2: log-likelihood with 2 hidden states = -28.8082438475
Action 1: log-likelihood with 4 hidden states = -58.4683156993
Action 4: log-likelihood with 3 hidden states = -6.80297992394
Action 3: log-likelihood with 3 hidden states = 3.09445083704
Action 1: log-likelihood with 5 hidden states = -58.8066245565
The best number of hidden-states for action 1 is: 3
Action 5: log-likelihood with 1 hidden states = 0.79173089881
Action 5: log-likelihood with 2 hidden states = 2.25366305162
Action 

[{1: 3, 2: 0, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 4, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 2, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 2, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 2}]

In [10]:
from joblib import Parallel, delayed
import multiprocessing
     
num_cores = multiprocessing.cpu_count()

actions = range(1,6)
    
Parallel(n_jobs=num_cores)(delayed(LOOcv)(action, 10, 1000) for action in actions)

Action 1: log-likelihood with 1 hidden states = -63.4932376436
Action 2: log-likelihood with 1 hidden states = -31.0731412366
Action 3: log-likelihood with 1 hidden states = -0.363394588714
Action 4: log-likelihood with 1 hidden states = -6.93741697547
Action 1: log-likelihood with 2 hidden states = -59.8213393624
Action 4: log-likelihood with 2 hidden states = 0.929071145707
Action 3: log-likelihood with 2 hidden states = 4.68990204463
Action 1: log-likelihood with 3 hidden states = -57.455598275
Action 2: log-likelihood with 2 hidden states = -28.8082370491
Action 1: log-likelihood with 4 hidden states = -58.4679774988
Action 4: log-likelihood with 3 hidden states = -6.80242306926
Action 3: log-likelihood with 3 hidden states = 3.09440675514
Action 1: log-likelihood with 5 hidden states = -58.8064768949
Action 4: log-likelihood with 4 hidden states = -1.62237631267
Action 2: log-likelihood with 3 hidden states = -26.7638080174
Action 3: log-likelihood with 4 hidden states = 1.6712060

[{1: 3, 2: 0, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 9, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 2, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 2, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 2}]

In [4]:
from joblib import Parallel, delayed
import multiprocessing
     
num_cores = multiprocessing.cpu_count()

actions = range(1,6)

# with an elevate numbe of hidden-states, I allow for a lower number of iterations to save time  
Parallel(n_jobs=num_cores)(delayed(LOOcv)(action, 20, 100) for action in actions)

Action 1: log-likelihood with 1 hidden states = -63.4932376436
Action 2: log-likelihood with 1 hidden states = -31.0731412366
Action 4: log-likelihood with 1 hidden states = -6.93741697547
Action 3: log-likelihood with 1 hidden states = -0.363394588714
Action 1: log-likelihood with 2 hidden states = -59.8213483636
Action 4: log-likelihood with 2 hidden states = 0.929071118155
Action 1: log-likelihood with 3 hidden states = -57.455572519
Action 3: log-likelihood with 2 hidden states = 4.68990224906
Action 2: log-likelihood with 2 hidden states = -28.8082459989
Action 1: log-likelihood with 4 hidden states = -58.4683336649
Action 4: log-likelihood with 3 hidden states = -6.81716310542
Action 3: log-likelihood with 3 hidden states = 3.09443636785
Action 1: log-likelihood with 5 hidden states = -58.7978794535
Action 4: log-likelihood with 4 hidden states = -1.62237688416
Action 2: log-likelihood with 3 hidden states = -26.8155143808
Action 1: log-likelihood with 6 hidden states = -59.70839

[{1: 3, 2: 0, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 16, 3: 0, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 16, 4: 0, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 2, 5: 0},
 {1: 0, 2: 0, 3: 0, 4: 0, 5: 20}]

In [ ]:
# NOTE the results above come from a second attempt of LOO, with a larger number of iteration.
# However I find more reasonable to continue using
# 1:13, 2:17, 3:17, 4:2, 5:2 

### Fit the 5 HMMs on the whole train partition

In [51]:
models={}
people = range(0,8)
best_hs = [13,17,17,2,2]
for act in range(1,6):
    print "training hmm for action %s" %(act)
    action_train = np.empty([0,5]) #5 is the num of dimensions for each obs 
    seqlenght_train = np.empty(0)
    for person in people:    
        action_ix = [i for i,j in enumerate(y_train[person][0]) if j == act]
        person_action = x_train[person][:,action_ix]
        action_train = np.append(action_train, np.transpose(person_action), axis=0)
        seqlenght_train = np.append(seqlenght_train, person_action.shape[1])
    model = GaussianHMM(n_components=best_hs[act-1],
                        covariance_type="diag",
                        n_iter=1000).fit(action_train, seqlenght_train.astype(int))
    models[act]=model

training hmm for action 1
training hmm for action 2
training hmm for action 3
training hmm for action 4
training hmm for action 5


### Testing on the same dataset I have trained the models...

In [53]:
y_pred = np.array([np.zeros(x_train[0].shape[1]),
                   np.zeros(x_train[1].shape[1]),
                   np.zeros(x_train[2].shape[1]),
                   np.zeros(x_train[3].shape[1]),
                   np.zeros(x_train[4].shape[1]),
                   np.zeros(x_train[5].shape[1]),
                   np.zeros(x_train[6].shape[1]),
                   np.zeros(x_train[7].shape[1])])
for i in range(8):
    print "predicting person %s" %(i+1)
    for j in range(x_train[i].shape[1]):
        for model in range(1,6):
            scores.append(models[model].score(np.transpose(x_train[i][:,j])))
        y_pred[i][j] = scores.index(max(scores))+1

predicting person 1
predicting person 2
predicting person 3
predicting person 4
predicting person 5
predicting person 6
predicting person 7
predicting person 8


In [54]:
for i in range(8):
    error = 0
    for j in range(len(y_pred[i])):
        # print (y_pred[i][j],y_train[i][0][j]) 
        if y_pred[i][j] != y_train[i][0][j]:
            error = error + 1 
    print 'Accuracy on person %s is: %s' %(i+1 , 1-error/len(y_pred[i]))

Accuracy on person 1 is: 0.908434821831
Accuracy on person 2 is: 0.89533431101
Accuracy on person 3 is: 0.888188073394
Accuracy on person 4 is: 0.785033097722
Accuracy on person 5 is: 0.880910165485
Accuracy on person 6 is: 0.833878464452
Accuracy on person 7 is: 0.904076207355
Accuracy on person 8 is: 0.915105053947


### Training and testing with another LOO

In [3]:
def LOOcv2(best_hs):
    people = range(0,8)
    splits = loo().split(people)
    for people_train, person_val in splits:
        print 'people train: %s, person validation: %s'%(people_train,person_val)
        models = {}
        # train
        action_train = np.empty([0,5]) #5 is the num of dimensions for each obs 
        seqlenght_train = np.empty(0)
        for action in range(1,6):    
            for person in people_train:
                action_ix = [i for i,j in enumerate(y_train[person][0]) if j == action]
                person_action = x_train[person][:,action_ix]
                action_train = np.append(action_train, np.transpose(person_action), axis=0)
                seqlenght_train = np.append(seqlenght_train, person_action.shape[1])
            # validation
            action_val = np.empty([0,5]) #5 is the num of dimensions for each obs 
            seqlenght_val = np.empty(0)
            val_action_ix = [i for i,j in enumerate(y_train[person_val[0]][0]) if j == action]
            val_person_action = x_train[person_val[0]][:,val_action_ix]
            action_val = np.append(action_val, np.transpose(val_person_action), axis=0)
            seqlenght_val = np.append(seqlenght_val, action_val.shape[1])
            # GaussianHMM
            print 'training HMM for action %s' %(action)
            models[action] = GaussianHMM(n_components= best_hs[action],
                                covariance_type="diag",
                                n_iter=10).fit(action_train, seqlenght_train.astype(int))
            # store the log-likelihood of the model on the validation partition
        y_pred = np.zeros(x_train[person_val[0]].shape[1])
        error = 0
        for j in range(x_train[person_val[0]].shape[1]):
            scores = []
            for action in range(1,6):
                scores.append( models[action].score(np.transpose(x_train[person_val[0]][:,j])) )
            y_pred[j] = scores.index(max(scores))+1 
            #print (y_pred[j], y_train[person_val[0]][0][j])
            if int(y_pred[j]) != y_train[person_val[0]][0][j]:
                error = error + 1
        print 'Accuracy on person %s is: %s' %(person_val[0], 1-error/len(y_pred))
        print classification_report(y_true=y_train[person_val[0]][0],y_pred=y_pred)
        print accuracy_score(y_pred=y_pred, y_true=y_train[person_val[0]][0])
        print '\n'

In [11]:
best_hs = {1:3,2:9,3:2,4:2,5:2}
LOOcv2(best_hs)

people train: [1 2 3 4 5 6 7], person validation: [0]
training HMM for action 1
training HMM for action 2
training HMM for action 3
training HMM for action 4
training HMM for action 5
Accuracy on person 0 is: 0.525823184484
             precision    recall  f1-score   support

          1       0.52      0.50      0.51       527
          2       0.72      0.58      0.64      3563
          3       0.95      0.74      0.83      5800
          4       0.14      0.03      0.05      5317
          5       0.30      1.00      0.46      2529

avg / total       0.55      0.53      0.50     17736

0.525823184484


people train: [0 2 3 4 5 6 7], person validation: [1]
training HMM for action 1
training HMM for action 2
training HMM for action 3
training HMM for action 4
training HMM for action 5
Accuracy on person 1 is: 0.643691271522
             precision    recall  f1-score   support

          1       0.56      0.71      0.63       657
          2       0.74      0.50      0.60      3666
 

In [5]:
best_hs = {1:13,2:17,3:2,4:2,5:2}
LOOcv2(best_hs)

people train: [1 2 3 4 5 6 7], person validation: [0]
training HMM for action 1
training HMM for action 2
training HMM for action 3
training HMM for action 4
training HMM for action 5
Accuracy on person 0 is: 0.516689219666
             precision    recall  f1-score   support

          1       0.38      0.38      0.38       527
          2       0.71      0.57      0.63      3563
          3       0.95      0.74      0.83      5800
          4       0.09      0.02      0.04      5317
          5       0.30      0.99      0.46      2529

avg / total       0.53      0.52      0.49     17736

0.516689219666


people train: [0 2 3 4 5 6 7], person validation: [1]
training HMM for action 1
training HMM for action 2
training HMM for action 3
training HMM for action 4
training HMM for action 5
Accuracy on person 1 is: 0.601922763565
             precision    recall  f1-score   support

          1       0.25      0.50      0.34       657
          2       0.51      0.35      0.42      3666
 

In [8]:
################
# FINAL MODELS #
################
models={}
people = range(0,8)
best_hs = [3,9,2,2,2]
for act in range(1,6):
    print "training hmm for action %s" %(act)
    action_train = np.empty([0,5]) #5 is the num of dimensions for each obs 
    seqlenght_train = np.empty(0)
    for person in people:    
        action_ix = [i for i,j in enumerate(y_train[person][0]) if j == act]
        person_action = x_train[person][:,action_ix]
        action_train = np.append(action_train, np.transpose(person_action), axis=0)
        seqlenght_train = np.append(seqlenght_train, person_action.shape[1])
    model = GaussianHMM(n_components=best_hs[act-1],
                        covariance_type="diag",
                        n_iter=10000).fit(action_train, seqlenght_train.astype(int))
    models[act]=model

training hmm for action 1
training hmm for action 2
training hmm for action 3
training hmm for action 4
training hmm for action 5


In [9]:
# y_test is an empt array that will be filled with predictions
predictions = np.array([np.zeros(x_test[0].shape[1]),np.zeros(x_test[1].shape[1])])

for i in [0,1]:
    for j in range(x_test[i].shape[1]):
        scores = [0]
        activity = 0
        for model in range(1,6):
            scores.append(models[model].score(np.transpose(x_test[i][:,j])))
            activity = model if scores[model] > scores[model-1] else activity
        predictions[i][j] = activity

In [76]:
predictions

array([array([ 3.,  3.,  3., ...,  4.,  4.,  4.]),
       array([ 2.,  3.,  3., ...,  3.,  3.,  3.])], dtype=object)

In [19]:
scipy.io.savemat('HAR_BadTestResults_hmm.mat',{'hmm_predictions':predictions})